<a href="https://colab.research.google.com/github/mvish7/PyTorch/blob/master/confusion_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

necessary imports

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

creating n/w architecture

In [0]:
class Network(nn.Module):
  def __init__(self):
    super(Network,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    
    self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)
    
  def forward(self,t):
    #implementing fwd pass
    
    # input layer
    t=t #returning the tensor as it as, no need of processing
    
    # hidden layer 1 
    t=self.conv1(t)
    t=F.relu(t)
    t=F.max_pool2d(t, kernel_size=2, stride=2)
    
    
    #hidden layer 2
    t=self.conv2(t)
    t=F.relu(t)
    t=F.max_pool2d(t, kernel_size=2, stride=2)
    
    #hidden layer 3
    t=t.reshape(-1,12*4*4)  #doing the flattening operation before passing the tensor from conv layer to the linear layer
    t=self.fc1(t)
    t=F.relu(t)
    
    #hidden layer 4
    t=self.fc2(t)
    t=F.relu(t)
    
    #output layer
    t=self.out(t)
    #t=F.softmax(t, dim=1)  no need to use softmax here, coz we are using cross entropy as our loss function and it has built in softmax.
    
    
    return t
  
      

In [0]:
model=Network()

creating training data object

In [3]:
trainset=torchvision.datasets.FashionMNIST(root='./data/FashionMNIST',
                                           train=True, 
                                           download=True,
               transform=transforms.Compose([transforms.ToTensor()]))

0it [00:00, ?it/s]

26427392it [00:02, 10630138.61it/s]                             


Extracting ./data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 73787.42it/s]                            
0it [00:00, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz


4423680it [00:01, 2974409.87it/s]                            
0it [00:00, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 24536.31it/s]            

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


this is the data loader instance

In [0]:
data_loader=torch.utils.data.DataLoader(trainset,batch_size=100, shuffle='true')
batch=next(iter(data_loader))
images, labels=batch

In [5]:
len(trainset)

60000

In [6]:
len(trainset.targets)

60000

setting the loss fun and optimizers

In [0]:
loss_fun= nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.01)

fun to find correct predixtions

In [0]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

let's train the n/w

In [13]:
for epoch in range(5):
  total_loss=0
  total_correct=0

  for batch in data_loader:
    images, labels= batch
    preds = model(images)
    loss = loss_fun(preds, labels)
  
    optimizer.zero_grad()
    loss.backward() #calculating the gradients
    optimizer.step() #update weights
  
    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)
  
  print("epoch:", epoch, "total correct:", total_correct, "loss:", total_loss)

epoch: 0 total correct: 46923 loss: 342.4742175042629
epoch: 1 total correct: 51446 loss: 232.04567217826843
epoch: 2 total correct: 51964 loss: 216.48347127437592
epoch: 3 total correct: 52478 loss: 205.33190037310123
epoch: 4 total correct: 52492 loss: 204.73954032361507


let's find accuracy

In [14]:
total_correct/len(trainset)

0.8748666666666667

getting the predictions from model over whole training set

In [0]:
def get_all_preds(model, data_loader):
  all_preds=torch.tensor([])
  for batch in data_loader:
    images, labels= batch
    
    preds = model(images)
    all_preds=torch.cat((all_preds, preds), dim=0)
    
    return all_preds
  

In [0]:
with torch.no_grad():
  prediction_loader=torch.utils.data.DataLoader(trainset, batch_size=60000)
  train_preds=get_all_preds(model, prediction_loader)

building confusion matrix now

these are the labels of whole trainset

In [27]:
trainset.targets

tensor([9, 0, 0,  ..., 3, 0, 5])

we only want the max num from the 10 predictions of each image, coz thats the actual right pred n/w has

In [28]:
train_preds.argmax(dim=1).shape

torch.Size([60000])

now lets make a pair of (label, pred)

In [0]:
stacked = torch.stack((trainset.targets,train_preds.argmax(dim=1) ), dim=1)

In [30]:
stacked.shape

torch.Size([60000, 2])

this is how it looks

In [31]:
stacked

tensor([[9, 9],
        [0, 0],
        [0, 0],
        ...,
        [3, 3],
        [0, 0],
        [5, 5]])

creating the list out of this stacked

In [33]:
stacked[0].tolist()

[9, 9]

exact configuration mat

In [34]:
cmt=torch.zeros(10,10, dtype=torch.int32)
cmt

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)

let's populate the values

In [0]:
for p in stacked:
  j,k= p.tolist()
  cmt[j,k]=cmt[j,k]+1

In [36]:
cmt

tensor([[5374,    4,   43,  144,   25,    6,  379,    0,   25,    0],
        [  15, 5809,    3,  135,   17,    5,   14,    0,    2,    0],
        [  81,    1, 4390,   51,  893,    3,  574,    1,    6,    0],
        [ 162,    5,   13, 5207,  538,    1,   61,    0,   12,    1],
        [  11,    2,  276,   52, 5347,    0,  308,    0,    4,    0],
        [   0,    0,    0,    0,    0, 5935,    1,   32,   10,   22],
        [1156,    5,  383,  167,  601,    3, 3661,    0,   24,    0],
        [   0,    0,    0,    0,    0,  220,    0, 5637,    2,  141],
        [  25,    2,   26,    9,   38,   32,  105,    2, 5761,    0],
        [   0,    0,    0,    2,    0,   98,    0,  153,    3, 5744]],
       dtype=torch.int32)

plotting the confusion matrix